# 深度玄学

## 前言

现在大家做毕设和论文也纷纷步入正轨了, 在调节网络参数的技巧和问题上也都会遇到自己的问题, 踩了自己应该踩的坑, 积累了自己独特的经验. 在下也是刚刚开始学习调整网络的结构, 所积累的调参经验非常有限, 也不敢班门弄斧. 以下部分的内容整理自最近学习到的一些课程文章, 很多地方自己也没有真正用到和体会到, 所以以下问题和内容是希望能够和大家一起来讨论以下, 希望可以抛砖引玉! 

注: 以下的所有的代码以tensorflow为例子.

## 1. 变量的初始化问题

网络参数的初始化主要有两个部分, 第一部分就是权重(weights)的初始化, 第二部分就是偏置量(bias)的初始化. 重点落在权重的初始化时候遇到的常见的问题上. 

### 1.1 权重(weights)全部初始化为0

In [ ]:
import tensorflow as tf
import numpy      as np 

num_input   = 512
n_hidden_1  = 1024
n_hidden_2  = 2048
num_classes = 10

weights = {
    'h1' : tf.Variable( tf.zeros( [num_input,  n_hidden_1 ] )),
    'h2' : tf.Variable( tf.zeros( [n_hidden_1, n_hidden_2 ] )),
    'out': tf.Variable( tf.zeros( [n_hidden_2, num_classes] ))
}


从上面的代码当中, 我们可以看出这只是一个最普通的三层全连接神经网络, 其中, 作者将所有的权值初始化为了0.

显然这样子的初始化是会有问题的.

问题一:  

- __这样子会存在什么样的问题?__

    [友情链接](https://zhuanlan.zhihu.com/p/27190255)

- __如果将0换成其他数字呢? 比如全部替换成为0.1__    
[友情链接](https://blog.csdn.net/bvl10101111/article/details/70787683)


### 1.2 权重(weights)全部初始化为正态分布的随机数

In [ ]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

上面的代码, 应该是我们比较常见的初始化方法. 该方法对每一层的参数都初始化为随机数了, 随机数的分布范围由其方差__0.1__确定好了. 
与上面的第一种初始化方法相比, 虽然在每一层面的的数据的各个核的初始化方法都一样, 但是由于每一个核都是用随机数进行初始化, 所以, 实际上每一个核的初始值都是不一样的, 顶多就是同分布而已.  
  

问题二:
``` python
tf.truncated_normal()
tf.random_normal()
```
**以上两个函数有什么区别呢?**   
[友情链接](https://blog.csdn.net/u013713117/article/details/65446361)

反正如果让我们自己手写一个卷积神经网络, 而不是修改别人的网络的话, 我猜有200%可能性大家都会选择使用上面的方法来对参数进行初始化.   
这种初始化参数的方法确实也没啥很大的毛病, 一般情况下该学习到的特征还是能够学习到, 该收敛的还是可以收敛, 但是我们可以做得更好. 

此话怎讲?

### 1.3 Xavier初始化方法

人们上世纪90年代在深层的神经网络中, 很容发生梯度消失和梯度爆炸的问题.     
- __梯度爆炸__  也就是在梯度传播的过程中, 使用链式法求导导数在传递的过程中被放地越来越大, 导致网络输出结果振荡非常严重, 无法收敛      
- __梯度消失__  也就是在梯度传播的过程中, 使用链式法求导导数在传递的过程中被放地越来越小, 导致网络参数更新非常缓慢, 学习不下去了      
 
 

 这些问题想必即便大家没有体会过也都是见过很多参考书说过了. 在这里重复提起的原因是, 通过良好的参数初始化我们可以很好地避免, 这个问题, 让我们的炼丹网络赢在起跑线上!


Xavier的初始化方法推导过程参见王老师给本科生上课用的ppt     ----------------->

__道理我们都懂了, 但是我们要怎样去实现这个方法呢?__

首先我们先来纯手工实现一个

In [ ]:
import tensorflow as tf

def xavier_init(fan_in, fan_out, constant = 1):
    low = -constant * np.sqrt(6.0 / (fan_in + fan_out))
    high = constant * np.sqrt(6.0 / (fan_in + fan_out))
    return tf.random_uniform((fan_in, fan_out),
                             minval=low, maxval=high, dtype=tf.float32)
def weight_variable(shape):
  initial = xavier_init(shape[0], shape[3])
  return tf.Variable(initial)

然而, 这个2010就提出来了的初始化方法tensorflow怎么可能会没有现有的实现呢? 从tf 0.8开始就支持了这种初始化方法, 虽然感觉我们用的很少, 但事实上这个方法确实能够很好加快我们训练速度. 获得更高的准确率. 

In [ ]:
W = tf.get_variable("W", shape=[784, 256],
           initializer=tf.contrib.layers.xavier_initializer())

### 1.4 MSRA初始化方法

但是问题又来了, 我们在王老师的ppt中讲到, 我们在深度神经网络中经常需要用到的 ` Relu ` 这种梯度无比平稳的激活函数, 然而这个函数并不符合` Xavier `初始化方法的假设, 那就是激活函数在0附近的导数约等于1, ` tanh `激活函数倒是蛮符合这个假设了. 所以, 使用` Xavier `初始化方法对于现有的深度网络模型来说也并不是完美的. 

青年才俊何凯明在2015年就提出了一种修正方案, 大佬的论文在[这里](https://arxiv.org/pdf/1502.01852.pdf), 其实, 这篇论文我也没有看, 就只看了一下别简单的推导过程. 估计大家听得看得也挺闷的,其基本的出发点和`Xavier`方法是一致的, 那就是尽量保持前后层之间的权重的方差是一致的, 就不推导了. 直接看一下在tensorflow里面是怎么样实现这一个过程的吧 :)

Ps. ` MSRA `初始化方法又叫 ` He initialization `, __这里的He是什么意思呢?__

In [ ]:
he_weights = {
        'h1': tf.get_variable('w1_he',    [num_input, n_hidden_1],
                              initializer=tf.contrib.layers.variance_scaling_initializer()),
        'h2': tf.get_variable('w2_he',    [n_hidden_1, n_hidden_2],
                              initializer=tf.contrib.layers.variance_scaling_initializer()),
        'out': tf.get_variable('wout_he', [n_hidden_2, num_classes],
                               initializer=tf.contrib.layers.variance_scaling_initializer())
    }

当然了, 这里面用到的tensorflow API `tf.contrib.layers.variance_scaling_initializer()`不仅仅可以实现何大神提出的初始化方法, 还可以有其他的参数选项, 可以参考下面的API说明. 其中, 默认的参数就是 `He initialization`

``` Python

# To get Delving Deep into Rectifiers (also know as the "MSRA initialization"), use (Default):
factor=2.0 mode='FAN_IN' uniform=False

# To get Convolutional Architecture for Fast Feature Embedding, use:
factor=1.0 mode='FAN_IN' uniform=True

# To get Understanding the difficulty of training deep feedforward neural networks, use:
factor=1.0 mode='FAN_AVG' uniform=True.

# To get xavier_initializer use either:
factor=1.0 mode='FAN_AVG' uniform=True, or
factor=1.0 mode='FAN_AVG' uniform=False.
``` 

特别需要说明的是, 在何大神论文中的实验__表明了当网络增加到33层之后，对比效果更加明显!__

### 1.5  偏置量(bias)的初始化问题 

没有太多需要注意的问题, 一般都直接初始化为0就可以了, 偶尔在一些门开关中需要设置为1, 但这多出现在自然语言处理上, 很少用到图像处理中!

## 2.  学习率的问题

传说中, Andrew Ng在调参的时候的个人经验和偏好是:
- 第一梯队： 
 - learning rate α    
- 第二梯队： 
 - hidden units                  
 - mini-batch size                  
 - momentum β     
- 第三梯队： 
 - number of layers                  
 - learning rate decay                  
 - other optimizer hyperparameters


作者：星晴      
链接：https://www.zhihu.com/question/29641737/answer/254462551    
来源：知乎     
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。   

Ng都如此这般, 可见学习率的初始化问题是多么重要.  
学习率的初始化问题能够直接影响到模型是否收敛及最终的收敛程度, 哪怕是我们使用的是动量方法和自适应的方法Adam. 我们在这里都要小心对待学习率的问题啊, 虽然我们的学习率调整算法会有很强的适应性, 但是如果开始的学习率设置大了, 动量的积累就会很大, 还是会给我们的网络最终结果带来不少影响.   

在这个问题上, 我们可以看看大佬们是怎么说的([点击这里](https://www.jiqizhixin.com/articles/nn-learning-rate))

``` Python
...build your model...
# Add the optimizer
train_op = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
# Add the ops to initialize variables.  These will include 
# the optimizer slots added by AdamOptimizer().
init_op = tf.initialize_all_variables()

# launch the graph in a session
sess = tf.Session()
# Actually intialize the variables
sess.run(init_op)
# now train your model
for ...:
  sess.run(train_op)
```

然而, 遗憾的是在学习率的初始化问题上却是没有太多的理论支持, 简单来说就是一门玄学, 一般初始化值为 `0.1 ~ 0.0001`, 而且这个数值也是我从网上找来的, 不同的人还有不同的说法   

若果你是仅仅想把网络结果跑一跑, 体会一下的话, 学习率的设置的问题可以不用太关注, 但是如果你是想把神经网络的准确度跑到极致的话, 就一定须让认真对待学习率的问题, 不断地进行尝试了.

下面第一个友情链接是伟鹏安利了的各种常见的梯度下降法的数学描述, 再次安利一下, 体会一把数学之美. 第二个链接则是Adam在tensorflow中的源代码实现.

- 友情链接1: [An overview of gradient descent optimization algorithms](http://ruder.io/optimizing-gradient-descent/)
- 友情链接2: [tensorflow adam source code](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/training/adam.py#L39)

## 3. 特殊的网络层

### 3.1 dropout层

In [ ]:
pool2 = np.arange(7*7*64*100).astype(float)
# Dense Layer
pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

>Again, inputs specifies the input tensor, which is the output tensor from our dense layer (dense).
>
>The rate argument specifies the dropout rate; here, we use 0.4, which means 40% of the elements will be randomly dropped out during training.
>
>The training argument takes a boolean specifying whether or not the model is currently being run in training mode; dropout will only be performed if training is True. Here, we check if the mode passed to our model function cnn_model_fn is TRAIN mode.

dropout 方法大家都熟悉, 不过是多加了一层而已.   
Dropout的作用有两个
- 一个就是打破不同kernal之间平衡性, 让不同的kernal能够学习到不同的特征
- 第二个就是王老师在课堂上讲到的, 屏蔽了网络的部分节点进行训练, 能够让单个神经元更加健壮, 降低了对其他神经元依赖, 能够有效地降低过拟合的事情.  


一般dropout的值都是 0.5, 也有设置为0.3或者0.7的, 一般在深层的神经网络中作用突出.    

问题3:   
__dropout节点有没有什么不好的地方?__    
__在预测阶段是否生效?__  
[友情链接](#问题3参考答案)

### 3.2 BN(batch_normalization)层

BN层就是对每一层都做一个batch正则化处理, 但是如果仅仅是做正则化处理的话, 是会使得网络损失已经学习的特征的, 所以, 除了做正则化处理之外, 还需要做添加两个可学习的参数`w`和`b`, 具体算法原理可参考一下两个友情链接:

[友情链接1: 论文](https://arxiv.org/pdf/1502.03167.pdf)    
[友情链接2: 博客](https://blog.csdn.net/fate_fjh/article/details/53375881)

传说中, 使用了BN的神经网络可以不太关注数据与处理时候的正则化问题; 传说中, 使用了BN的卷积神经网络再也不用担心梯度爆炸和梯度消失的问题; 传说中, 使用了BN的神经网络甚至可以不用太关心学习率的问题; 传说中, 使用了BN的卷积神经网络的训练速度飞快, 妈妈再也不用担心我的破电脑.... 

In [ ]:
# 使用tensorflow 中的一个高层封装的BN代码例子
def model_fn(features, labels, mode):
    # ...  neural network layers ...
    logits = tf.layers.dense(Y4, 200, use_bias=False)
    bn = tf.layers.batch_normalization(logits,
        axis=1,
        center=True,
        scale=False,
        training=(mode == tf.estimator.ModeKeys.TRAIN))
    Y5 = tf.nn.relu(bn)
    # ...  more neural network layers ...

上述代码的详细说明在[这里](https://github.com/martin-gorner/tensorflow-mnist-tutorial/blob/master/README_BATCHNORM.md),     

从上面中样例代码中可以看到, BN处理应该是在将数据输入激活函数之前的一个步骤, 他的位置不能在激活函数之后. 原因我也忘了.

## 4. 正则化项

不应该因为害怕出现过拟合而使用小网络。相反，应该进尽可能使用大网络，然后使用正则化技巧来控制过拟合。

## 参考代码

给出以下的样例代码是因为这个github仓库里面的代码已经囊括了大多数的初始化, 正则化的方法, 可供大家使用和参考   

样例代码github地址:
    https://github.com/hwalsuklee/tensorflow-mnist-MLP-batch_normalization-weight_initializers/blob/master/run_main.py

``` Python   

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os

from six.moves import urllib

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.layers.python.layers import batch_norm as batch_norm

SOURCE_URL = 'http://yann.lecun.com/exdb/mnist/'
DATA_DIRECTORY = "data"
LOGS_DIRECTORY = "logs/train"

# train params
training_epochs = 15
batch_size = 100
display_step = 50

# network params
n_input = 784
n_hidden_1 = 256
n_hidden_2 = 256
n_classes = 10

# Store layers weight & bias

with tf.name_scope('weight'):
    normal_weights = {
        'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1]),name='w1_normal'),
        'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]),name='w2_normal'),
        'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]),name='wout_normal')
    }
    truncated_normal_weights  = {
        'h1': tf.Variable(tf.truncated_normal([n_input, n_hidden_1],stddev=0.1),name='w1_truncated_normal'),
        'h2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2],stddev=0.1),name='w2_truncated_normal'),
        'out': tf.Variable(tf.truncated_normal([n_hidden_2, n_classes],stddev=0.1),name='wout_truncated_normal')
    }
    xavier_weights  = {
        'h1': tf.get_variable('w1_xaiver', [n_input, n_hidden_1],initializer=tf.contrib.layers.xavier_initializer()),
        'h2': tf.get_variable('w2_xaiver', [n_hidden_1, n_hidden_2],initializer=tf.contrib.layers.xavier_initializer()),
        'out': tf.get_variable('wout_xaiver',[n_hidden_2, n_classes],initializer=tf.contrib.layers.xavier_initializer())
    }
    he_weights = {
        'h1': tf.get_variable('w1_he', [n_input, n_hidden_1],
                              initializer=tf.contrib.layers.variance_scaling_initializer()),
        'h2': tf.get_variable('w2_he', [n_hidden_1, n_hidden_2],
                              initializer=tf.contrib.layers.variance_scaling_initializer()),
        'out': tf.get_variable('wout_he', [n_hidden_2, n_classes],
                               initializer=tf.contrib.layers.variance_scaling_initializer())
    }
with tf.name_scope('bias'):
    normal_biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden_1]),name='b1_normal'),
        'b2': tf.Variable(tf.random_normal([n_hidden_2]),name='b2_normal'),
        'out': tf.Variable(tf.random_normal([n_classes]),name='bout_normal')
    }
    zero_biases = {
        'b1': tf.Variable(tf.zeros([n_hidden_1]),name='b1_zero'),
        'b2': tf.Variable(tf.zeros([n_hidden_2]),name='b2_zero'),
        'out': tf.Variable(tf.zeros([n_classes]),name='bout_normal')
    }
weight_initializer = {'normal':normal_weights, 'truncated_normal':truncated_normal_weights, 'xavier':xavier_weights, 'he':he_weights}
bias_initializer = {'normal':normal_biases, 'zero':zero_biases}

# user input
from argparse import ArgumentParser

WEIGHT_INIT = 'xavier'
BIAS_INIT = 'zero'
BACH_NORM = True

def build_parser():
    parser = ArgumentParser()
    parser.add_argument('--weight-init',
                        dest='weight_initializer', help='weight initializer',
                        metavar='WEIGHT_INIT', required=True)
    parser.add_argument('--bias-init',
                        dest='bias_initializer', help='bias initializer',
                        metavar='BIAS_INIT', required=True)
    parser.add_argument('--batch-norm',
                        dest='batch_normalization', help='boolean for activation of batch normalization',
                        metavar='BACH_NORM', required=True)
    return parser

# Download the data from Yann's website, unless it's already here.
def maybe_download(filename):
    if not tf.gfile.Exists(DATA_DIRECTORY):
        tf.gfile.MakeDirs(DATA_DIRECTORY)
    filepath = os.path.join(DATA_DIRECTORY, filename)
    if not tf.gfile.Exists(filepath):
        filepath, _ = urllib.request.urlretrieve(SOURCE_URL + filename, filepath)
        with tf.gfile.GFile(filepath) as f:
            size = f.size()
        print('Successfully downloaded', filename, size, 'bytes.')
    return filepath

# Batch normalization implementation
# from https://github.com/tensorflow/tensorflow/issues/1122
def batch_norm_layer(inputT, is_training=True, scope=None):
    # Note: is_training is tf.placeholder(tf.bool) type
    return tf.cond(is_training,
                    lambda: batch_norm(inputT, is_training=True,
                    center=True, scale=True, activation_fn=tf.nn.relu, decay=0.9, scope=scope),
                    lambda: batch_norm(inputT, is_training=False,
                    center=True, scale=True, activation_fn=tf.nn.relu, decay=0.9,
                    scope=scope, reuse = True))

# Create model of MLP with batch-normalization layer
def MLPwithBN(x, weights, biases, is_training=True):
    with tf.name_scope('MLPwithBN'):
        # Hidden layer with RELU activation
        layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
        layer_1 = batch_norm_layer(layer_1,is_training=is_training, scope='layer_1_bn')
        layer_1 = tf.nn.relu(layer_1)
        # Hidden layer with RELU activation
        layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
        layer_2 = batch_norm_layer(layer_2, is_training=is_training, scope='layer_2_bn')
        layer_2 = tf.nn.relu(layer_2)
        # Output layer with linear activation
        out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Create model of MLP without batch-normalization layer
def MLPwoBN(x, weights, biases):
    with tf.name_scope('MLPwoBN'):
        # Hidden layer with RELU activation
        layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
        layer_1 = tf.nn.relu(layer_1)
        # Hidden layer with RELU activation
        layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
        layer_2 = tf.nn.relu(layer_2)
        # Output layer with linear activation
        out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# main function
def main():
    # Parse argument
    parser = build_parser()
    options = parser.parse_args()
    weights = weight_initializer[options.weight_initializer]
    biases = bias_initializer[options.bias_initializer]
    batch_normalization = options.batch_normalization

    # Import data
    mnist = input_data.read_data_sets('data/', one_hot=True)

    # Boolean for MODE of train or test
    is_training = tf.placeholder(tf.bool, name='MODE')

    # tf Graph input
    x = tf.placeholder(tf.float32, [None, 784])
    y_ = tf.placeholder(tf.float32, [None, 10]) #answer

    # Predict
    if batch_normalization=='True':
        y = MLPwithBN(x,weights,biases,is_training)
    else:
        y = MLPwoBN(x, weights, biases)

    # Get loss of model
    with tf.name_scope("LOSS"):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y,y_))

    # Define optimizer
    with tf.name_scope("ADAM"):
        train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

    # moving_mean and moving_variance need to be updated
    if batch_normalization == "True":
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        if update_ops:
            train_ops = [train_step] + update_ops
            train_op_final = tf.group(*train_ops)
        else:
            train_op_final = train_step

    # Get accuracy of model
    with tf.name_scope("ACC"):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    # Create a summary to monitor loss tensor
    tf.scalar_summary('loss', loss)

    # Create a summary to monitor accuracy tensor
    tf.scalar_summary('acc', accuracy)

    # Merge all summaries into a single op
    merged_summary_op = tf.merge_all_summaries()

    # Add ops to save and restore all the variables
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer(), feed_dict={is_training: True})

    # Training cycle
    total_batch = int(mnist.train.num_examples / batch_size)

    # op to write logs to Tensorboard
    summary_writer = tf.train.SummaryWriter(LOGS_DIRECTORY, graph=tf.get_default_graph())

    # Loop for epoch
    for epoch in range(training_epochs):

        # Loop over all batches
        for i in range(total_batch):

            batch = mnist.train.next_batch(batch_size)

            # Run optimization op (backprop), loss op (to get loss value)
            # and summary nodes
            _, train_accuracy, summary = sess.run([train_op_final, accuracy, merged_summary_op] , feed_dict={x: batch[0], y_: batch[1], is_training: True})

            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)

            # Display logs
            if i % display_step == 0:
                print("Epoch:", '%04d,' % (epoch + 1),
                "batch_index %4d/%4d, training accuracy %.5f" % (i, total_batch, train_accuracy))

    # Calculate accuracy for all mnist test images
    print("test accuracy for the latest result: %g" % accuracy.eval(
    feed_dict={x: mnist.test.images, y_: mnist.test.labels, is_training: False}))

if __name__ == '__main__':
    main()
```


#### 问题3参考答案


王老师在上课的时候提到了以下两点:
- 增加了dropout节点之后, 会使得整个网络的训练速度下降, 可能需要增加两道三倍的训练时间  
- dropout节点只在训练的时候用到, 预测的时候不应该再对网络中的节点进行屏蔽. 倒是应该将每个节点的输出值乘上mask的概率, 这样子能够使得网络输出保持应有的数学期望.
